In [1]:
%cd ..

/home/teo/userdata/virtuosoNet


In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
from virtuoso.train import prepare_dataloader
from virtuoso.parser import get_parser
from virtuoso import utils, model as modelzoo
from virtuoso.dataset import ScorePerformDataset, FeatureCollate
import random
import torch

In [13]:
parser = get_parser()
args = parser.parse_args(
    args=["--yml_path=ymls/han_measnote.yml",
          "--data_path=datasets/main_dataset_clamped",
          "--emotion_data_path=datasets/emotion_dataset_clamped",
          "--device=cpu"]
)
args, net_params, configs = utils.handle_args(args)
device = 'cpu'

In [14]:
checkpoint_path = '../virtuosonet_checkpoints/yml_path=ymls/han_measnote.yml data_path=main_dataset_clamped emotion_data_path=emotion_dataset_clamped meas_note=True iters_per_checkpoint=300 delta_weight=5.0 delta_loss=True vel_balance_loss=True intermediate_loss=False graph_keys=[]_220112-153119/checkpoint_last.pt'
# checkpoint_path = '../virtuosonet_checkpoints/yml_path=ymls/isgn_beatmeas_x.yml meas_note=True batch_size=24 iters_per_checkpoint=300 len_valid_slice=1200 delta_weight=5.0 delta_loss=True vel_balance_loss=True_220118-210647/checkpoint_last.pt'


In [5]:
from pathlib import Path
args.yml_path = next(Path(checkpoint_path).parent.glob('*.yml'))
net_param = torch.load(checkpoint_path, map_location='cpu')['network_params']


In [6]:
args.graph_keys

[]

In [7]:
model = modelzoo.make_model(net_params)

In [8]:
model = utils.load_weight(model, checkpoint_path)
model.to(device)
model.eval()

=> loaded checkpoint '../virtuosonet_checkpoints/yml_path=ymls/han_measnote.yml data_path=main_dataset_clamped emotion_data_path=emotion_dataset_clamped meas_note=True iters_per_checkpoint=300 delta_weight=5.0 delta_loss=True vel_balance_loss=True intermediate_loss=False graph_keys=[]_220112-153119/checkpoint_last.pt' (epoch 77)


VirtuosoNet(
  (score_encoder): HanEncoder(
    (note_fc): Sequential(
      (0): Linear(in_features=79, out_features=128, bias=True)
    )
    (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (voice_net): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (beat_attention): ContextAttention(
      (attention_net): Linear(in_features=512, out_features=512, bias=True)
    )
    (beat_rnn): LSTM(512, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (measure_attention): ContextAttention(
      (attention_net): Linear(in_features=256, out_features=256, bias=True)
    )
    (measure_rnn): LSTM(256, 128, batch_first=True, bidirectional=True)
  )
  (performance_encoder): HanPerfEncoder(
    (performance_measure_attention): ContextAttention(
      (attention_net): Linear(in_features=128, out_features=128, bias=True)
    )
    (performance_encoder): LSTM(128, 64, num_layers=2, batch_first=True, b

In [9]:
from torch.utils.data import DataLoader

hier_type = ['is_hier', 'in_hier', 'hier_beat', 'hier_meas', 'meas_note']
curr_type = [x for x in hier_type if getattr(args, x)]

train_set = ScorePerformDataset(args.data_path, 
                                type="train", 
                                len_slice=args.len_slice, 
                                len_graph_slice=args.len_graph_slice, 
                                graph_keys=args.graph_keys, 
                                hier_type=curr_type)

train_loader = DataLoader(train_set, batch_size=16, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())
small_train_loader = DataLoader(train_set, batch_size=5, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())
tiny_train_loader = DataLoader(train_set, batch_size=1, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())


In [10]:
random.seed(0)
train_set.update_slice_info()
batch = next(iter(train_loader))
batch_x, batch_y, beat_y, meas_y, note_locations, align_matched, pedal_status, edges = utils.batch_to_device(batch, device)

model.eval()
outputs, perform_mu, perform_var, total_out_list = model(batch_x, batch_y, edges, note_locations)
# with torch.no_grad():
#   score_embedding = model.score_encoder(batch_x, edges, note_locations)

TypeError: exceptions must derive from BaseException

In [ ]:
%cd ..

%load_ext autoreload
%autoreload 2
# import sys

# sys.path.append('../')



from virtuoso.train import prepare_dataloader
from virtuoso.parser import get_parser
from virtuoso import utils, model as modelzoo
from virtuoso.dataset import ScorePerformDataset, FeatureCollate
import random
import torch

parser = get_parser()
args = parser.parse_args(
    args=["--yml_path=ymls/han_measnote.yml",
          "--data_path=datasets/main_dataset_clamped",
          "--emotion_data_path=datasets/emotion_dataset_clamped",
          "--device=cpu"]
)
args, net_params, configs = utils.handle_args(args)
device = 'cpu'

checkpoint_path = '../virtuosonet_checkpoints/yml_path=ymls/han_measnote.yml data_path=main_dataset_clamped emotion_data_path=emotion_dataset_clamped meas_note=True iters_per_checkpoint=300 delta_weight=5.0 delta_loss=True vel_balance_loss=True intermediate_loss=False graph_keys=[]_220112-153119/checkpoint_last.pt'
# checkpoint_path = '../virtuosonet_checkpoints/yml_path=ymls/isgn_beatmeas_x.yml meas_note=True batch_size=24 iters_per_checkpoint=300 len_valid_slice=1200 delta_weight=5.0 delta_loss=True vel_balance_loss=True_220118-210647/checkpoint_last.pt'


from pathlib import Path
args.yml_path = next(Path(checkpoint_path).parent.glob('*.yml'))
net_param = torch.load(checkpoint_path, map_location='cpu')['network_params']


args.graph_keys

model = modelzoo.make_model(net_params)

model = utils.load_weight(model, checkpoint_path)
model.to(device)
model.eval()

model.network_params.num_edge_types

from torch.utils.data import DataLoader

hier_type = ['is_hier', 'in_hier', 'hier_beat', 'hier_meas', 'meas_note']
curr_type = [x for x in hier_type if getattr(args, x)]

train_set = ScorePerformDataset(args.data_path, 
                                type="train", 
                                len_slice=args.len_slice, 
                                len_graph_slice=args.len_graph_slice, 
                                graph_keys=args.graph_keys, 
                                hier_type=curr_type)

train_loader = DataLoader(train_set, batch_size=16, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())
small_train_loader = DataLoader(train_set, batch_size=5, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())
tiny_train_loader = DataLoader(train_set, batch_size=1, shuffle=False, num_workers=args.num_workers, pin_memory=args.pin_memory, collate_fn=FeatureCollate())


random.seed(0)
train_set.update_slice_info()
batch = next(iter(train_loader))
batch_x, batch_y, beat_y, meas_y, note_locations, align_matched, pedal_status, edges = utils.batch_to_device(batch, device)

# model.eval()
# # outputs, perform_mu, perform_var, total_out_list = model(batch_x, batch_y, edges, note_locations)
# with torch.no_grad():
#   score_embedding = model.score_encoder(batch_x, edges, note_locations)